In [1]:
import torch
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import lsh_py_wrapper as pyLSH
import knn_py_wrapper as pyKNN

In [2]:
# Input file, first column are labels, rest are values for dimensions
data_file = "../data/100exp10d.csv"

# Number of intermediate checkpoints to graph
checkpoints = 6

# cuda for GPU, cpu for CPU
mode = torch.device('cuda')

In [3]:
# Parameters
learn_rate = 1e-4
iterations = 10000
loss_margin = 50            # Margin for Contrastive Loss
momentum = 0.9              # Gradient Descent Momentum
pair_selection = "knn"      # random, knn, or lsh

In [4]:
# Select random pairs of points in the final space and return Euclidean distance and if labels matched
def rand_pairs(data, labels):
    perm = torch.randperm(data.size(0))
    equals = torch.eq(labels, labels[perm]).squeeze().to(torch.float)
    dists = torch.sum((data - data[perm]) ** 2, 1)
    return equals, dists

In [5]:
# Use GPU KNN to select furthest points of the same group
def knn_pairs(data, labels, it, mode):
    # Get furthest points of the same group
    # indices where label is 0 or 1
    ind0 = torch.where(labels == 0)[0]
    ind1 = torch.where(labels == 1)[0]
    # data points where label is 0 or 1
    data0 = data[ind0]
    data1 = data[ind1]
    # convert to numpy representation formatted for KNN lib
    ref0 = data0.cpu().detach().numpy().flatten("F")
    ref1 = data1.cpu().detach().numpy().flatten("F")
    ref0_size = int(len(ref0) / 3)
    ref1_size = int(len(ref1) / 3)
    # indices for k-neighbors for sets of points with label 0 or 1
    knn0 = list(pyKNN.gpu_knn(ref0, ref0, 3, ref0_size))
    knn1 = list(pyKNN.gpu_knn(ref1, ref1, 3, ref1_size))
    # indices in the sets of points with label 0 or 1, corresponding to furthest point of that set
    furthest0 = knn0[-ref0_size:]
    furthest1 = knn1[-ref1_size:]
    dists0 = torch.sum((data0 - data0[furthest0]) ** 2, 1)
    dists1 = torch.sum((data1 - data1[furthest1]) ** 2, 1)
    dists = torch.cat((dists0, dists1), 0)
    equals = torch.ones(labels.shape)
    if mode == torch.device('cuda'):
        equals = equals.cuda()
        dists = dists.cuda()
    return equals, dists

In [6]:
# Use GPU LSH to select approximate hard examples
# scales better than KNN since no need to compare every pair
def lsh_pairs(data, labels, it, hash_ptr):
    #TODO complete this function
    raise NotImplementedError
    #LSH parameters
    refresh = 10
    num_tables = 5
    projection_dim = 3
    bucket_wid = 3
    probe_bins = 3
    
    # Prepare Data
    data = data.cpu().detach().numpy().flatten().tolist()[:9]
    
    # Call to LSH lib
    if it % refresh == 0:
        hash_ptr = pyLSH.lsh_hash(hash_ptr, 3, num_tables, projection_dim, bucket_wid, data)
    ind = pyLSH.lsh_search(hash_ptr, 2, probe_bins, len(data), data, data)
    ind = torch.tensor(np.asarray(list(ind)))    

In [7]:
# Plot and save loss over all iterations
# file name determined by what pair selection method is used
def graph_loss(losses, pair_selection):
    x = range(0, 100 * len(losses), 100)
    y = losses
    plt.plot(x, y)
    plt.xlabel("Iterations")
    plt.ylabel("Loss")
    plt.title("With " + pair_selection + " pairs")
    plt.ylim(ymin=0)
    plt.savefig('../results/' + pair_selection + '_pairs_loss.png')

In [8]:
# Visualize Neural Net output in 3D space
def graph_points(points, labels, fname):
    points = points.cpu().detach().numpy()
    x1 = points[:, 0]
    x2 = points[:, 1]
    x3 = points[:, 2]
    y = np.squeeze(labels.cpu().detach().numpy())

    colors = ['red', 'blue']
    fig = plt.figure()
    ax = Axes3D(fig)
    ax.scatter(x1, x2, x3, c=y, cmap=matplotlib.colors.ListedColormap(colors))
    plt.savefig('../results/checkpoint_' + fname)

In [9]:
# Reading Data
data = []
labels = []
for ind, line in enumerate(open(data_file)):
    labels.append(np.array(int(line[0])))
    data.append(np.array(np.fromstring(line.split(',', 1)[1], sep = ',')))
    
data = np.column_stack(data)
labels = np.column_stack(labels)

In [10]:
# Change Numpy matrices to torch tensors of type Float
labels = torch.tensor(labels.transpose()).to(torch.float)
data = torch.tensor(data.transpose()).to(torch.float)

# Move to GPU if mode is CUDA
if mode == torch.device('cuda'):
    data = data.cuda()
    labels = labels.cuda()

In [11]:
# Define layer widths
D_in, D_1, D_2, D_3, D_4, D_out = data.shape[1], 100, 50, 25, 10, 3

# Define Model Architecture
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, D_1),
    torch.nn.ReLU(),
    torch.nn.Linear(D_1, D_2),
    torch.nn.ReLU(),
    torch.nn.Linear(D_2, D_3),
    torch.nn.ReLU(),
    torch.nn.Linear(D_3, D_4),
    torch.nn.ReLU(),
    torch.nn.Linear(D_4, D_out),
)
# Put on gpu if mode cuda
if mode == torch.device('cuda'):
    model.cuda()

# Define Optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learn_rate, momentum=momentum)

In [ ]:
print("Commence Learning!!!")
accumulator = 0
losses = []
lsh_hash_ptr = 0
checkpoint_count = 0
easy_learning = 1
for it in range(iterations):
    # Zero gradients
    optimizer.zero_grad()
    
    # Forward Pass
    results = model(data)
    
    # Loss Calculation
    if easy_learning or pair_selection == 'random':    
        if_same, euclidean_dist = rand_pairs(results, labels)
    elif pair_selection == 'knn':
        if_same, euclidean_dist = knn_pairs(results, labels, it, mode)
    elif pair_selection == 'lsh':
        if_same, euclidean_dist, lsh_hash = lsh_pairs(results, labels, it, lsh_hash_ptr)
        
    loss = torch.mean((if_same) * torch.pow(euclidean_dist, 2) +
            (1 - if_same) * torch.pow(torch.clamp(loss_margin - euclidean_dist, min=0), 2))
    
    accumulator += loss.item()

    # Backward Pass
    loss.backward()
    optimizer.step()
    
    # Print averaged losses to keep track of progress
    if it % 100 == 0:
        if it == 0:
            print("Iteration:", it, "\t|\tInitial Loss:", accumulator)
            losses.append(accumulator)
        else:
            avg_loss = accumulator / 100
            print("Iteration:", it, "\t|\tAverage Loss (last 100):", avg_loss)
            losses.append(avg_loss)
            # If average loss drops below threshold, begin using hard negative mining
            if avg_loss < 10 and easy_learning:
                graph_points(results, labels, "Easy_Learning_Final")
                easy_learning = 0
                if pair_selection != 'random':
                    print("Switch pair selection from random to", pair_selection)
        accumulator = 0
    
    # Create checkpoing graphs to view feature space evolution
    if it % round(iterations / (checkpoints + 1)) == 0:
        graph_points(results, labels, pair_selection + "_" + str(checkpoint_count))
        checkpoint_count += 1    

Commence Learning!!!
Iteration: 0 	|	Initial Loss: 1149.986572265625
Iteration: 100 	|	Average Loss (last 100): 1249.9820727539063
Iteration: 200 	|	Average Loss (last 100): 1260.9751812744141
Iteration: 300 	|	Average Loss (last 100): 1244.9631427001952
Iteration: 400 	|	Average Loss (last 100): 1220.4373046875
Iteration: 500 	|	Average Loss (last 100): 1241.349203491211
Iteration: 600 	|	Average Loss (last 100): 1244.5103076171874
Iteration: 700 	|	Average Loss (last 100): 991.3421710205079
Iteration: 800 	|	Average Loss (last 100): 876.3463287353516
Iteration: 900 	|	Average Loss (last 100): 457.63023230969907
Iteration: 1000 	|	Average Loss (last 100): 0.30582932472229
Switch pair selection from random to knn
Iteration: 1100 	|	Average Loss (last 100): 0.3246704824268818
Iteration: 1200 	|	Average Loss (last 100): 0.07235890179872513
Iteration: 1300 	|	Average Loss (last 100): 0.042749574668705465
Iteration: 1400 	|	Average Loss (last 100): 0.02961947599425912
Iteration: 1500 	|	Av

In [ ]:
graph_points(results, labels, pair_selection + "_final")

In [ ]:
graph_loss(losses, pair_selection)